In [169]:
# !pip install cassandra-driver

In [172]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

In [178]:
session.execute("use dev")
session.execute("CREATE TABLE users (id int PRIMARY KEY, address text)")

In [32]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window


In [3]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages com.datastax.spark:spark-cassandra-connector_2.11:2.5.0 --conf spark.cassandra.connection.host=127.0.0.1 pyspark-shell"

In [18]:
spark : SparkSession = SparkSession.builder\
    .master( "local[*]" ) \
    .appName( "Cassandra-spark-example" ) \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .getOrCreate()

In [9]:
data = spark\
    .read\
    .options(header='true', inferSchema='true')\
    .csv("file:///Users/yashiro/Desktop/spark_connect_examples/inputdata/locations.csv")\
    .withColumn("id", monotonically_increasing_id().astype("int"))

In [10]:
data.columns

['countriesAndTerritories',
 'location',
 'continent',
 'population_year',
 'population',
 'id']

In [149]:
# data.coalesce(1).write.options(header='true').csv("file:///Users/yashiro/Desktop/spark_connect_examples/inputdata/countries.csv")

In [12]:
# ls /Users/yashiro/Desktop/spark_connect_examples/inputdata/countries.csv

In [13]:
# """create keyspace dev with replication = {'class':'SimpleStrategy','replication_factor':1};"""

In [14]:
# """use dev"""

In [15]:
# """
# CREATE TABLE populations(
#    id int PRIMARY KEY,
#    location text,
#    continent text,
#    population_year int,
#    population int
#    );
# """

In [20]:
cols = ['id', 'location', 'continent', 'population_year', 'population'] #'countriesAndTerritories',
data.select(*cols).write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="populations", keyspace="dev")\
    .save()

In [22]:
cass_data = spark.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(table="populations", keyspace="dev")\
    .load()

In [23]:
cass_data.printSchema()

root
 |-- id: integer (nullable = false)
 |-- continent: string (nullable = true)
 |-- location: string (nullable = true)
 |-- population: integer (nullable = true)
 |-- population_year: integer (nullable = true)



In [28]:
cass_data.show()

+---+-------------+--------------+----------+---------------+
| id|    continent|      location|population|population_year|
+---+-------------+--------------+----------+---------------+
|123|       Africa|    Mauritania|   4649660|           2020|
| 54|       Europe|       Denmark|   5792203|           2020|
| 75|       Europe|       Germany|  83783945|           2020|
|189|       Africa|          Togo|   8278737|           2020|
| 15|         Asia|    Bangladesh| 164689383|           2020|
| 98|         Asia|        Israel|   8655541|           2020|
| 49|North America|          Cuba|  11326616|           2020|
| 10|      Oceania|     Australia|  25499881|           2020|
|131|       Africa|       Morocco|  36910558|           2020|
| 36|       Africa|    Cape Verde|    555988|           2020|
| 40|       Africa|          Chad|  16425859|           2020|
|198|       Africa|      Tanzania|  59734213|           2020|
| 46|North America|    Costa Rica|   5094114|           2020|
| 43|Sou

In [30]:
cass_data.groupBy("continent").count().show()

+-------------+-----+
|    continent|count|
+-------------+-----+
|       Europe|   51|
|       Africa|   54|
|         null|    1|
|North America|   36|
|South America|   13|
|      Oceania|    8|
|         Asia|   46|
+-------------+-----+



In [50]:
window = Window.partitionBy("continent")

In [55]:
max_df = cass_data.withColumn("max_population", max("population").over(window))\
  .filter(col("max_population") == col("population")).orderBy("continent")

In [56]:
max_df.show()

+---+-------------+-------------+----------+---------------+--------------+
| id|    continent|     location|population|population_year|max_population|
+---+-------------+-------------+----------+---------------+--------------+
|141|       Africa|      Nigeria| 206139587|           2020|     206139587|
| 42|         Asia|        China|1439323774|           2020|    1439323774|
|158|       Europe|       Russia| 145934460|           2020|     145934460|
|200|North America|United States| 331002647|           2020|     331002647|
| 10|      Oceania|    Australia|  25499881|           2020|      25499881|
| 27|South America|       Brazil| 212559409|           2020|     212559409|
+---+-------------+-------------+----------+---------------+--------------+

